# Stokes Sinker

Demonstration example for setting up particle swarms with different material properties. This system consists of a dense, high viscosity sphere falling through a background lower density and viscosity fluid.


$$
\nabla \cdot
        \color{Blue}{\underbrace{\Bigl[
                \boldsymbol{\tau} -  p \mathbf{I} \Bigr]}_{\mathbf{F}}} =
        \color{Maroon}{\underbrace{\Bigl[ \mathbf{f} \Bigl] }_{\mathbf{h}}}
$$

$$
\underbrace{\Bigl[ \nabla \cdot \mathbf{u} \Bigr]}_{\mathbf{h}_p} = 0
$$

In [1]:
#|  echo: false  # Hide in html version

# This is required to fix pyvista 
# (visualisation) crashes in interactive notebooks (including on binder)

import nest_asyncio
nest_asyncio.apply()

In [2]:
#| output: false # Suppress warnings in html version

import underworld3 as uw
import numpy as np
import sympy

[MacBook-Pro.local:31565] shmem: mmap: an error occurred while determining whether or not /var/folders/0j/bnxlsh897sl6b1rv06fnt5r80000gp/T//ompi.MacBook-Pro.502/jf.0/148766720/sm_segment.MacBook-Pro.502.8de0000.0 could be created.


#### Setup scaling of the model

In [3]:
u = uw.scaling.units

### make scaling easier
ndim = uw.scaling.non_dimensionalise
nd   = uw.scaling.non_dimensionalise
dim  = uw.scaling.dimensionalise 

velocity = 1.0 * u.centimeter / u.hour
model_length = 2. * u.meter
model_height = 1. * u.meter
refViscosity = 1e6 * u.pascal * u.second
bodyforce = 200 * u.kilogram / u.metre**3 * 9.81 * u.meter / u.second**2

KL = model_height
Kt = KL / velocity
KM = bodyforce * KL**2 * Kt**2



scaling_coefficients  = uw.scaling.get_coefficients()
scaling_coefficients["[length]"] = KL
scaling_coefficients["[time]"]= Kt
scaling_coefficients["[mass]"]= KM

scaling_coefficients

[mass],254275200000000.0 kilogram
[length],1.0 meter
[temperature],1.0 kelvin
[time],360000.0 second
[substance],1.0 mole


#### Setup the mesh

In [4]:
# mesh = uw.meshing.UnstructuredSimplexBox(
#             minCoords=(nd(0. * u.centimeter), nd(0. * u.centimeter)), 
#             maxCoords=(nd(20. * u.centimeter), nd(10. * u.centimeter)),
#             cellSize=1 / 12
# )

mesh = uw.meshing.StructuredQuadBox(
            minCoords=(nd(-1. * u.meter), nd(-50. * u.centimeter)), 
            maxCoords=(nd(1. * u.meter), nd(50. * u.centimeter)),
            elementRes=(32,32)
)

x, y = mesh.X

mesh.view()

Structured box element resolution 32 32


Mesh # 0: .meshes/uw_structuredQuadBox_minC(-1.0, -0.5)_maxC(1.0, 0.5).msh

No variables are defined on the mesh

| Boundary Name            | ID    | Min Size | Max Size |
| ------------------------------------------------------ |
| Bottom                   | 11    | 63       | 63       |
| Top                      | 12    | 63       | 63       |
| Right                    | 13    | 63       | 63       |
| Left                     | 14    | 63       | 63       |
| Null_Boundary            | 666   | 1089     | 1089     |
| All_Boundaries           | 1001  | 128      | 128      |
| All_Boundaries           | 1001  | 128      | 128      |
| UW_Boundaries            | --    | 1469     | 1469     |
| ------------------------------------------------------ |


DM Object: uw_.meshes/uw_structuredQuadBox_minC(-1.0, -0.5)_maxC(1.0, 0.5).msh 1 MPI process
  type: plex
uw_.meshes/uw_structuredQuadBox_minC(-1.0, -0.5)_maxC(1.0, 0.5).msh in 2 dimensions:
  

In [5]:
from mpi4py import MPI

if MPI.COMM_WORLD.size == 1:
    
    import pyvista as pv
    import underworld3.visualisation as vis

    pvmesh = vis.mesh_to_pv_mesh(mesh)

    pl = pv.Plotter(window_size=(1000, 500), shape=(1, 1))


    pl.add_mesh(
        pvmesh,
        cmap="coolwarm",
        edge_color="Black",
        show_edges=True,
        use_transparency=False,
        opacity=1,
        show_scalar_bar=True,
    )

    pl.show()


Widget(value='<iframe src="http://localhost:61283/index.html?ui=P_0x15bb9f400_0&reconnect=auto" class="pyvista…

In [6]:
# mesh variables
p_soln = uw.discretisation.MeshVariable(varname="p", mesh=mesh, num_components=1, degree=0, continuous=False )
v_soln = uw.discretisation.MeshVariable(varname="v", mesh=mesh, num_components=2, degree=1)


### The Stokes solver

There are a number of pre-defined *solver systems* defined in `underworld3` 
which are templates for orchestrating the underlying PETSc objects. 
A solver requires us to define the unknown in the form of `meshVariables`, 
provide boundary conditions, a constitutive model, 
and provide `uw.functions` to define constitutive
properties, and driving terms.

We will use the `Stokes` solver to solve for the body force (forcing term) and viscous drag (flux term).

The solver classes themselves are documented, so we can figure out what 
is needed before we define anything:


In [7]:
uw.systems.Stokes.view()


This class provides functionality for a discrete representation
of the Stokes flow equations assuming an incompressibility
(or near-incompressibility) constraint.

$$
\nabla \cdot
        \color{Blue}{\underbrace{\Bigl[
                \boldsymbol{\tau} -  p \mathbf{I} \Bigr]}_{\mathbf{F}}} =
        \color{Maroon}{\underbrace{\Bigl[ \mathbf{f} \Bigl] }_{\mathbf{h}}}
$$

$$
\underbrace{\Bigl[ \nabla \cdot \mathbf{u} \Bigr]}_{\mathbf{h}_p} = 0
$$

The flux term is a deviatoric stress ( $\boldsymbol{\tau}$ ) related to velocity gradients
  ( $\nabla \mathbf{u}$ ) through a viscosity tensor, $\eta$, and a volumetric (pressure) part $p$

$$
    \mathbf{F}: \quad \boldsymbol{\tau} = \frac{\eta}{2}\left( \nabla \mathbf{u} + \nabla \mathbf{u}^T \right)
$$

The constraint equation, $\mathbf{h}_p = 0$ gives incompressible flow by default but can be set
to any function of the unknown  $\mathbf{u}$ and  $\nabla\cdot\mathbf{u}$

## Properties

  - The unknowns are velocities $\mathbf{u}$ and a pressure-like constraint parameter $\mathbf{p}$

  - The viscosity tensor, $\boldsymbol{\eta}$ is provided by setting the `constitutive_model` property to
one of the scalar `uw.constitutive_models` classes and populating the parameters.
It is usually a constant or a function of position / time and may also be non-linear
or anisotropic.

  - $\mathbf f$ is a volumetric source term (i.e. body forces)
  and is set by providing the `bodyforce` property.

  - An Augmented Lagrangian approach to application of the incompressibility
constraint is to penalise incompressibility in the Stokes equation by adding
$ \lambda \nabla \cdot \mathbf{u} $ when the weak form of the equations is constructed.
(this is in addition to the constraint equation, unlike in the classical penalty method).
This is activated by setting the `penalty` property to a non-zero floating point value which adds
the term in the `sympy` expression.

  - A preconditioner is usually required for the saddle point system and this is provided
though the `saddle_preconditioner` property. The default choice is $1/\eta$ for a scalar viscosity function.

## Notes

  - For problems with viscoelastic behaviour, the flux term contains the stress history as well as the
    stress and this term is a Lagrangian quantity that has to be tracked on a particle swarm.

  - The interpolation order of the `pressureField` variable is used to determine the integration order of
the mixed finite element method and is usually lower than the order of the `velocityField` variable.

  - It is possible to set discontinuous pressure variables by setting the `p_continous` option to `False`



### Add in a swarm for multimaterials
This can be used to assign different materials different properties within the domain

In [9]:
swarm = uw.swarm.Swarm(mesh=mesh)
material = uw.swarm.IndexSwarmVariable("M", swarm, indices=2)
swarm.populate_petsc(fill_param=2)

In [12]:
lightMaterial = 0
heavyMaterial = 1

In [13]:
with swarm.access(material):
    material.data[...] = lightMaterial

    inside = (swarm.data[:, 0] - center[0]) ** 2 + (swarm.data[:, 1] - center[1]) ** 2 < radius**2
    material.data[inside] = heavyMaterial

In [14]:
from mpi4py import MPI

if MPI.COMM_WORLD.size == 1:
    
    import pyvista as pv
    import underworld3.visualisation as vis

    pvmesh = vis.mesh_to_pv_mesh(mesh)
    pvswarm = vis.swarm_to_pv_cloud(swarm)
    with swarm.access(material):
        pvswarm.point_data["M"] = material.data

    pl = pv.Plotter(window_size=(1000, 500), shape=(1, 1))


    pl.add_mesh(
        pvswarm,
        cmap="coolwarm",
        edge_color="Black",
        show_edges=True,
        scalars="M",
        use_transparency=False,
        opacity=1,
        show_scalar_bar=True,
    )

    pl.show()

Widget(value='<iframe src="http://localhost:61283/index.html?ui=P_0x325fce110_1&reconnect=auto" class="pyvista…

##### Add in a passive tracer to track the sinker

In [10]:
center=np.array([0.,nd(30.*u.centimeter)]) 
radius= nd(10. * u.centimeter)

In [11]:
tip_tracer = uw.swarm.Swarm(mesh=mesh)
tip_tracer.add_particles_with_coordinates(np.array([[center[0], center[1]-radius]]))

2

### Problem setup

In [15]:
# Setup stokes
stokes = uw.systems.Stokes(mesh, velocityField=v_soln, pressureField=p_soln)

### Constitutive Models

Most of the solvers require a constitutive model to be provided and its
parameters populated. This is to allow flexibility in defining / redefining 
solvers during a model calculation.

We need a viscous flow model. We can look at the documentation first. 

In [16]:
uw.constitutive_models.ViscousFlowModel.view()


### Viscous Flow Model

$$\tau_{ij} = \eta_{ijkl} \cdot \frac{1}{2} \left[ \frac{\partial u_k}{\partial x_l} + \frac{\partial u_l}{\partial x_k} \right]$$

where $\eta$ is the viscosity, a scalar constant, `sympy` function, `underworld` mesh variable or
any valid combination of those types. This results in an isotropic (but not necessarily homogeneous or linear)
relationship between $\tau$ and the velocity gradients. You can also supply $\eta_{IJ}$, the Mandel form of the
constitutive tensor, or $\eta_{ijkl}$, the rank 4 tensor.

The Mandel constitutive matrix is available in `viscous_model.C` and the rank 4 tensor form is
in `viscous_model.c`.




In [17]:
stokes.constitutive_model = uw.constitutive_models.ViscousFlowModel

##### We use the material index to map a viscosity

In [18]:
eta_l = uw.function.expression(
                    r'\eta_{l}',
                    sym=nd(1e6*u.pascal*u.second),
                    description="light material viscosity"
                        )

eta_h = uw.function.expression(
                    r'\eta_{h}',
                    sym=nd(1e6*u.pascal*u.second),
                    description="light material viscosity"
                        )

In [19]:
viscosity_fn = eta_l * material.sym[0] + eta_h * material.sym[1]

In [20]:
viscosity_fn

{ \eta_{h} \hspace{ 0.04pt } }*{M^{[1]}}(N.x, N.y) + { \eta_{l} \hspace{ 0.03pt } }*{M^{[0]}}(N.x, N.y)

In [21]:
stokes.constitutive_model.Parameters.shear_viscosity_0 = viscosity_fn
stokes.saddle_preconditioner = 1 / viscosity_fn
stokes.constitutive_model.view()

**Class**: <class 'underworld3.constitutive_models.ViscousFlowModel'>

This consititutive model is formulated for 2 dimensional equations

<IPython.core.display.Latex object>

##### We repeat this to map the density to each material

In [22]:
rho_l = uw.function.expression(
                    r'\rho_{l}',
                    sym=nd(10*u.kilogram/u.meter**3),
                    description="light material density"
                        )

rho_h = uw.function.expression(
                    r'\rho_{h}',
                    sym=nd(500*u.kilogram/u.meter**3),
                    description="heavy material density"
                        )


gravity = uw.function.expression(
                    r'g',
                    sym=nd(9.81*u.meter/u.second**2),
                    description="gravity"
                        )

In [23]:
density = rho_l * material.sym[0] + rho_h * material.sym[1]

In [24]:
stokes.bodyforce = sympy.Matrix([0, -1*density*gravity])

In [25]:
stokes.view()

**Class**: <class 'underworld3.systems.solvers.SNES_Stokes'>

### Saddle point system solver

Primary problem: 

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

#### Boundary Conditions

| Type   | Boundary | Expression | 
|:------------------------ | -------- | ---------- | 


This solver is formulated as a 2 dimensional problem with a 2 dimensional mesh

### Setting up boundary coniditions
Free slip everywhere

In [26]:
v_0 = uw.function.expression(
                    r'v_{0}',
                    sym=0.,
                    description="zero velocity component"
)

In [36]:
# Add boundary conditions
stokes.add_essential_bc([None, v_0], "Bottom")
stokes.add_essential_bc([None, v_0], "Top"  )

stokes.add_essential_bc([v_0, None], "Left")
stokes.add_essential_bc([v_0, None], "Right"  )




#### Check the system setup by using view()

In [37]:
stokes.view()

**Class**: <class 'underworld3.systems.solvers.SNES_Stokes'>

### Saddle point system solver

Primary problem: 

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

#### Boundary Conditions

| Type   | Boundary | Expression | 
|:------------------------ | -------- | ---------- | 
| **essential** | Bottom | $\left[\begin{matrix}\infty & { v_{0} \hspace{ 0.14pt } }\end{matrix}\right]  $ | 
| **essential** | Top | $\left[\begin{matrix}\infty & { v_{0} \hspace{ 0.14pt } }\end{matrix}\right]  $ | 
| **essential** | Left | $\left[\begin{matrix}{ v_{0} \hspace{ 0.14pt } } & \infty\end{matrix}\right]  $ | 
| **essential** | Right | $\left[\begin{matrix}{ v_{0} \hspace{ 0.14pt } } & \infty\end{matrix}\right]  $ | 
| **essential** | Bottom | $\left[\begin{matrix}\infty & 0\end{matrix}\right]  $ | 
| **essential** | Top | $\left[\begin{matrix}\infty & 0\end{matrix}\right]  $ | 
| **essential** | Left | $\left[\begin{matrix}0 & \infty\end{matrix}\right]  $ | 
| **essential** | Right | $\left[\begin{matrix}0 & \infty\end{matrix}\right]  $ | 


This solver is formulated as a 2 dimensional problem with a 2 dimensional mesh

##### Solve the system

In [29]:
stokes.petsc_options["snes_converged_reason"] = None

In [30]:
time = 0.
step = 0

In [31]:
# Solve the system n times
for nsteps in range(5):
    with tip_tracer.access():
        print(f'step = {step}, time = {dim(time, u.minute)}, tip tracer y = {dim(tip_tracer.data[:,1][0], u.centimeter)}')
    
    stokes.solve(zero_init_guess=True)
    dt = stokes.estimate_dt()
    ### advect particles
    swarm.advection(v_soln.sym, dt)
    tip_tracer.advection(v_soln.sym, dt)
    
    step += 1
    time += dt

step = 0, time = 0.0 minute, tip tracer y = 20.0 centimeter
  Nonlinear Solver_14_ solve converged due to CONVERGED_FNORM_RELATIVE iterations 2
step = 1, time = 36.650014189196376 minute, tip tracer y = 16.93671751248592 centimeter
  Nonlinear Solver_14_ solve converged due to CONVERGED_FNORM_RELATIVE iterations 2
step = 2, time = 70.55728364869803 minute, tip tracer y = 13.939965135344371 centimeter
  Nonlinear Solver_14_ solve converged due to CONVERGED_FNORM_RELATIVE iterations 2
step = 3, time = 101.93454371328794 minute, tip tracer y = 10.972377685641858 centimeter
  Nonlinear Solver_14_ solve converged due to CONVERGED_FNORM_RELATIVE iterations 2
step = 4, time = 132.83746932526347 minute, tip tracer y = 8.010296853594744 centimeter
  Nonlinear Solver_14_ solve converged due to CONVERGED_FNORM_RELATIVE iterations 2


In [32]:
from mpi4py import MPI

if MPI.COMM_WORLD.size == 1:
    
    import pyvista as pv
    import underworld3.visualisation as vis

    pvmesh = vis.mesh_to_pv_mesh(mesh)
    pvmesh.point_data["V"] = uw.visualisation.vector_fn_to_pv_points(pvmesh, v_soln.sym)

    velocity_points = uw.visualisation.meshVariable_to_pv_cloud(v_soln)
    velocity_points.point_data["V"] = uw.visualisation.vector_fn_to_pv_points(velocity_points, v_soln.sym)


    pvswarm = vis.swarm_to_pv_cloud(swarm)
    with swarm.access(material):
        pvswarm.point_data["M"] = material.data

    pl = pv.Plotter(window_size=(1000, 500), shape=(1, 1))


    pl.add_mesh(
        pvswarm,
        cmap="coolwarm",
        edge_color="Black",
        show_edges=True,
        scalars="M",
        use_transparency=False,
        opacity=1,
        show_scalar_bar=True,
    )

    arrows = pl.add_arrows(velocity_points.points, velocity_points.point_data["V"], mag=1e-1, opacity=0.1, show_scalar_bar=False)

    pl.show()

Widget(value='<iframe src="http://localhost:61283/index.html?ui=P_0x32dc432e0_2&reconnect=auto" class="pyvista…